In [18]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
from io import StringIO
import requests
import unicodedata

def remove_special_characters(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))

def download_csv_and_show_table():
    source_url = source_entry.get()
    col_index = col_index_entry.get()

    csv_stream = download_csv(source_url)

    if csv_stream:
        try:
            df = pd.read_csv(csv_stream, on_bad_lines='skip')
        except pd.errors.ParserError as e:
            print(f"Erro ao ler o CSV: {e}")
            return

        if col_index:
            col_index = int(col_index)
            if col_index < len(df.columns):
                df = df.iloc[:, [col_index]]
            else:
                print("Índice de coluna fora do intervalo")
                return
        else:
            df = df.iloc[:, :4]

        for col in df.columns:
            df[col] = df[col].apply(lambda x: remove_special_characters(str(x).upper()))

        # Create a window to display the table
        data_window = tk.Toplevel(root)
        data_window.title("Dados CSV")
        data_tree = ttk.Treeview(data_window)
        data_tree["columns"] = df.columns
        data_tree.heading("#0", text="Índice")
        for col_id, col_name in enumerate(df.columns):
            data_tree.heading(col_id, text=col_name)
            data_tree.column(col_id, width=100)
        for index, row in df.iterrows():
            data_tree.insert("", "end", text=index, values=[row[i] for i in range(len(df.columns))])
        data_tree.pack()

        # Create combobox filters for each column
        column_comboboxes = []

        def apply_filters():
            filtered_df = df.copy()
            for col_name, combobox in column_comboboxes:
                selected_value = combobox.get()
                if selected_value:
                    filtered_df = filtered_df[filtered_df[col_name] == selected_value]

            # Clear the existing rows in the Treeview
            for row in data_tree.get_children():
                data_tree.delete(row)

            # Repopulate the Treeview with the filtered data
            for index, row in filtered_df.iterrows():
                data_tree.insert("", "end", text=index, values=[row[i] for i in range(len(filtered_df.columns))])

        for col_name in df.columns:
            unique_values = df[col_name].unique()
            combobox_label = ttk.Label(data_window, text=f"Filter {col_name}:")
            combobox_label.pack()
            combobox = ttk.Combobox(data_window, values=unique_values)
            combobox.pack()
            column_comboboxes.append((col_name, combobox))

        apply_filters_button = ttk.Button(data_window, text="Apply Filters", command=apply_filters)
        apply_filters_button.pack()

def download_csv(url):
    response = requests.get(url)
    if response.status_code == 200:
        return StringIO(response.text)
    else:
        return None

# Main window configuration
root = tk.Tk()
root.title("Aplicativo CSV")

# Create widgets
source_label = ttk.Label(root, text="URL do CSV:")
source_entry = ttk.Entry(root, width=40)
col_index_label = ttk.Label(root, text="Índice da Coluna (opcional):")
col_index_entry = ttk.Entry(root, width=5)
fetch_button = ttk.Button(root, text="Carregar e Mostrar Tabela", command=download_csv_and_show_table)

# Position widgets
source_label.grid(row=0, column=0, sticky="w")
source_entry.grid(row=0, column=1, columnspan=2, sticky="w")
col_index_label.grid(row=1, column=0, sticky="w")
col_index_entry.grid(row=1, column=1, sticky="w")
fetch_button.grid(row=1, column=2)

# Start the GUI
root.mainloop()